# Balance packaging and recyclability

## 1. Read data

### 1.1. Upload **file**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving OutputMaterials.xlsx to OutputMaterials (3).xlsx


### 1.2. Import & load sheets

In [ ]:
import pandas as pd
import io
mat = pd.read_excel(io.BytesIO(uploaded ['OutputMaterials.xlsx']),sheet_name='material')
fr = pd.read_excel(io.BytesIO(uploaded ['OutputMaterials.xlsx']),sheet_name='food_req',header=0, index_col=0)
dim_mat = mat.shape
dim_fr = fr.shape

### 1.3. Food type selection

In [ ]:
Food_Type = "Fresh cheese" #@param ['Butter','Carbonated drinks','Coffees/Teas',"Cured meats",'Fish high fat','Fresh cheese',"High fat products", 'Juice','Milk','Most Fruits','Most Vegetables','Powder high fat','Ready meals','Red meats',"Snack foods"]
ft_prop = fr.loc[Food_Type]

In [ ]:
ft_prop = fr.loc[Food_Type]
ft_prop

O2          High
CO2         High
H2O         High
Light       High
Aroma     Medium
Grease    Medium
Name: Fresh cheese, dtype: object

## 2. Functions

In [ ]:
from numpy import maximum
# --------------------------------------------------------------------
#   Get selected food type requiremetns    
# --------------------------------------------------------------------
def GetParameters(ft_prop):
  params = []
  y = 0
  for x in ft_prop:
    if (x == 'Low'):    y = 5
    if (x == 'Medium'): y = 3
    if (x  == 'High'):  y = 1
    params.append(y)
  return params

# --------------------------------------------------------------------
#   Materials with similar properties     
# --------------------------------------------------------------------
def MaterialsWithSimilarProperties (params,din_mat,mat): 
  i = 0
  rango     = []
  oxygenbw  = []
  oxlist    = []
  aguabw    = []
  aglist    = []
  aromabw   = []
  arlist    = []
  grasabw   = []
  grlist    = []
  body      = []
  barrier   = []
  adhesive  = []
  for x in params:
    if (params[i] == 1 or params[i] == 2): 
      rango.append(1)
      rango.append(2)
    if (params[i] == 3): 
      rango.append(3)
      rango.append(3)
    if (params[i] == 4 or params[i] == 5): 
      rango.append(4)
      rango.append(5)
    i = i+1
  for x in range(dim_mat[0]):
    if mat.Oxygen[x] == rango[0] or mat.Oxygen[x] == rango[1]: 
      oxygenbw.append(mat.Material[x])
      oxlist.append(x)
    if mat.Water[x] == rango[2] or mat.Water[x] == rango[3]: 
      aguabw.append(mat.Material[x])
      aglist.append(x)
    if mat.Aroma[x] == rango[8] or mat.Aroma[x] == rango[9]: 
      aromabw.append(mat.Material[x])
      arlist.append(x)
    if mat.Grease[x] == rango[10] or mat.Grease[x] == rango[11]: 
      grasabw.append(mat.Material[x])
      grlist.append(x)
    if mat.BR[x] == 1:
      body.append(mat.Material[x])
    if mat.BR[x] == 3:
      barrier.append(mat.Material[x]) 
    if mat.BR[x] == 5:
      adhesive.append(mat.Material[x])   
  return oxygenbw,oxlist,aguabw,aglist,aromabw,arlist,grasabw,grlist,body,barrier,adhesive

# --------------------------------------------------------------------
#   WASTE STREAM FILTER      
# --------------------------------------------------------------------
def WasteStreamFilter (oxlist,aglist,arlist,grlist):
  oxygen = []
  agua   = []  
  aroma  = []
  grasa  = []
  for i in oxlist:
    if mat.WS[i] < 4:
      oxygen.append(mat.Material[i])
  for i in aglist:
    if mat.WS[i] < 4:
      agua.append(mat.Material[i])
  for i in arlist:
    if mat.WS[i] < 4:
      aroma.append(mat.Material[i])
  for i in grlist:
    if mat.WS[i] < 4:
      grasa.append(mat.Material[i])
  return oxygen,agua,aroma,grasa

# --------------------------------------------------------------------
#   WASTE STREAM FILTER      
# --------------------------------------------------------------------
def WSFilter (barrier,mat):
  b_nr = []
  b_ws = []
  bar = []
  x = 0
  y = 0
  for i in mat.Material:
    if i in barrier:
      b_nr.append(x)
      b_ws.append(mat.WS[x])
    x += 1
  for i in barrier:
    if b_ws[y] < 4:
      bar.append(i)
    y += 1
  return bar

# --------------------------------------------------------------------
#   REPEATED MATERIALS     
# --------------------------------------------------------------------
def RepeatedMaterials (oxygen,agua,aroma,grasa):
  matlist = []
  times   = []
  for i in oxygen:
    matlist.append(i)
    times.append(1)
  for i in agua:
    if i in matlist:
      times[matlist.index(i)] += 1
    else :
      matlist.append(i)
      times.append(1)
  for i in aroma:
    if i in matlist:
      times[matlist.index(i)] += 1
    else :
      matlist.append(i)
      times.append(1)
  for i in grasa:
    if i in matlist:
      times[matlist.index(i)] += 1
    else :
      matlist.append(i)
      times.append(1)
  return matlist,times


# --------------------------------------------------------------------
#   FIND & PRINT SOLUTION      
# --------------------------------------------------------------------
def PrintSol (sol):
  print("\n ************************************************************************************* \n")
  print("Congratulations, you have reached the beggining of the solution. Base materials:", sol)
  print("\n ************************************************************************************* \n")
  print("Now, a barrier will be needed for fullfiling the required criteria, threfore, it will be checked if an adhesive is needed too.")
  
def BodyMat (matlist, times, body,num):
  j = 0
  sol = []
  for i in matlist:
    if times[j] == num :
      if i in body:
        if i == 'BOPET':
          print("For BOPET -> Focus on rigid instead of flexible.")
        else:
          sol.append(i)   
    j += 1
  return sol
  
def FindSol (matlist,times,body):
  print(body)
  num = 4 
  sol = 0
  while num > 1:
    sol = BodyMat(matlist, times, body, num)
    if bool(sol) != 0:
      num =0
    else: num -=1 
  if bool(sol) == 0:
      print("No material available") # -> We'll see what to do here.
  else: PrintSol(sol)
  return sol

def find_nr (sol,mat):
  sol_nr = []
  sol_resin = []
  x = 0
  for i in mat.Material:
    if i in sol:
      sol_nr.append(x)
      sol_resin.append(mat.Resin[x])
    x += 1
  return [sol_nr,sol_resin]

# --------------------------------------------------------------------
#   MISSING ATTRIBUTES FOR EACH FOUND SOLUTION      
# --------------------------------------------------------------------
def MissingAtr (sol,oxygen,agua,aroma,grasa,params):
  missing = []
  for i in sol:
    if (i in oxygen) == 0:
      if params[0] < 4:
        missing.append("ox")
      else: print("No need to add barrier to oxygen for:", i)
    if (i in agua) == 0:
      if params[2] < 4:
        missing.append("ag")
      else: print("No need to add barrier to water for:",i)
    if (i in aroma) == 0:
      if params[4] < 4:
        missing.append("ar")
      else: print("No need to add barrier to aroma for:",i)
    if (i in grasa) == 0:
      if params[5] < 4:
        missing.append("gr")
      else: print("No need to add barrier to grease for:",i)
    missing.append(i)
  return missing 

# --------------------------------------------------------------------
#   POSSIBLE BARRIERS FOR EACH SOLUTION    
# --------------------------------------------------------------------
def PossibleBarriers (sol, oxygen, agua, aroma, grasa, barrier):
  n = 0
  a = 0
  total = []
  barr = []
  params = []
  for i in sol:
    j = missing[a]
    while j != i:
      total.append([]) 
      total[a].append(j)
      total[a].append([])
      if n == 0:
        barr.append(i)
        params.append(i)
        n =1
      if j == "ox":
        params.append(j)
        barr.append(j)
        for k in oxygen:
          if k in barrier:
            barr.append(k)
      if j == "ag":
        params.append(j)
        barr.append(j)
        for k in agua:
          if k in barrier:
            barr.append(k)
      if j == "ar":
        params.append(j)
        barr.append(j)
        for k in aroma:
          if k in barrier:
            barr.append(k)
      if j == "gr":
        params.append(j)
        barr.append(j)
        for k in grasa:
          if k in barrier:
            barr.append(k)
      a += 1
      j = missing[a]
    n = 0 
  return [barr, params]

# --------------------------------------------------------------------
# SPLIT BARRIERS
# --------------------------------------------------------------------
def split_bar (barr, sol):
# Get first body, and store -> Read until we reach 
  A = []
  b1 = 0
  m = 0
  n = 0
  # A[0][1]
  for i in barr:
    if i in sol:
      if i != b1:
         b1 = i
  return 0

# --------------------------------------------------------------------
#   GET RANGE   
# --------------------------------------------------------------------
def GetRange (params):
  i = 0
  rango     = []
  oxygenbw  = []
  oxlist    = []
  aguabw    = []
  aglist    = []
  aromabw   = []
  arlist    = []
  grasabw   = []
  grlist    = []
  body      = []
  barrier   = []
  adhesive  = [] 
  for x in params:
    if (params[i] == 1 or params[i] == 2): 
      rango.append(1)
      rango.append(2)
    if (params[i] == 3): 
      rango.append(1)
      rango.append(3)
    if (params[i] == 4 or params[i] == 5): 
      rango.append(1)
      rango.append(5)
    i = i+1
  return rango


# --------------------------------------------------------------------
#   REPEATED MATERIALS   
# --------------------------------------------------------------------
def Repeated (mats,n_params,params_2_match):
  times = []
  matlist = []
  for i in mats:
    if i in matlist:
       times[matlist.index(i)] += 1
    else:
      matlist.append(i)
      times.append(1)
  if n_params == max(times): times.append(7)
  return [matlist, times]

# --------------------------------------------------------------------
#   PRINT FINAL SOLUTION 
# --------------------------------------------------------------------
def PrintSolFinal (ml,t,body,mat,n_sol,n_resin,params_2_match):
  mx = max(t)
  a = 0
  barlist = []
  comments = []
  total = []
  if mx == 7:
    print("Solution found that fullfilled all barrier chriteria", params_2_match,"for:" ,body)
    t.pop()
    mx = max(t)
    for i in ml:
      if t[ml.index(i)] == mx:
        barlist.append(ml[a])
        if n_resin == "PE":
          cmnt = comments.append(mat.PE[n_sol])
          if cmnt != 'nan':
            aux = "For:" , ml[a], "beware that" , mat.PE[n_sol]
            total.append(aux)
        if n_resin == "PP":
          cmnt = comments.append(mat.PP[n_sol])
          if cmnt != 'nan':
            aux = "For:",ml[a], "beware that",mat.PP[n_sol]
            total.append(aux)
      a += 1
  else:
    mx = max(t)
    for i in ml:
      if t[ml.index(i)] == mx:
        barlist.append(ml[a])
        if n_resin == "PE":
          comments.append(mat.PE[n_sol])
          aux = "For:",ml[a], "beware that",mat.PE[n_sol]
          total.append(aux)
        if n_resin == "PP":
          comments.append(mat.PP[n_sol])
          aux = "For:",ml[a], "beware that",mat.PP[n_sol]
          total.append(aux)
      a += 1
  print("Possible barriers:", barlist) 
  if total: print(total)
  else: print("No additional comments required for this solution.")
  print( " ------------------------------ ")  
  return 0
    
# --------------------------------------------------------------------
#   REPEATED BARRIERS   
# --------------------------------------------------------------------
def RepeatedBarr (sol,barr,mat,sol_nr,sol_resin,params_list):
  params = ['ox','ag','ar','gr']
  n_body = -1
  n_params = 0
  a = 1
  params_2_match = []
  aux = []
  mats = []
  final = len(barr)
  for i in barr:
    if i in sol or a == final:
      if n_body > -1:
        n_sol = sol_nr[n_body]
        r_sol = sol_resin[n_body]
        if mats:
          [mlist , times] = Repeated(mats,n_params,params_2_match)
          psol = PrintSolFinal (mlist,times,body,mat,n_sol,r_sol,params_2_match)
      mats = []
      params_2_match = [] 
      body = i       
      n_body += 1
      n_params = 0
    elif i in params:
      params_2_match.append(i)
      n_params += 1
    else:
      mats.append(i)
    a += 1
  return aux

## 3. Run rest

In [ ]:
food_type_parameters = GetParameters(ft_prop)

### Materials before waste stream

In [ ]:
[oxygen_bws,ox_list,water_bws,wt_list,aroma_bws,ar_list,grease_bws,gr_list,body_mats,barrier_mats,adhesive_mats] = MaterialsWithSimilarProperties(food_type_parameters,dim_mat,mat)

### Materials after waste stream

In [ ]:
[oxygen, water, aroma, grease] = WasteStreamFilter(ox_list,wt_list,ar_list,gr_list)
[materials_list, times] = RepeatedMaterials(oxygen,water,aroma,grease)

Oxygen AWS: 	 ['ACR', 'PVOH', 'EVOH32', 'EVOH44', 'AlOx', 'SiOx']
Water  AWS: 	 ['MDPE', 'ACR', 'EVOH32', 'EVA', 'EMA']
Aroma  AWS: 	 ['PVOH', 'EVOH32', 'EVOH44', 'PETg']
Grease AWS: 	 ['HDPE', 'COC', 'PETg', 'EAA']
['ACR', 'PVOH', 'EVOH32', 'EVOH44', 'AlOx', 'SiOx', 'MDPE', 'EVA', 'EMA', 'PETg', 'HDPE', 'COC', 'EAA']
[2, 2, 3, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1]


### Solution

In [ ]:
sol = FindSol(materials_list,times,body_mats)
if 'BOPET' in sol:
  print("For BOPET -> Focus on rigid instead of flexible.")
if sol:
  missing = MissingAtr(sol,oxygen,water,aroma,grease,food_type_parameters) # Missing attributes
  barriers_WSF = WSFilter(barrier_mats,materials_list)  # Waste Stream filter for possible barriers
  [barriers,barrier_parameters] = PossibleBarriers(sol, oxygen, water, aroma, grease, barriers_WSF)
  [sol_nr,sol_resin] = find_nr(sol,materials_list)
  a = RepeatedBarr(sol,barriers,materials_list,sol_nr,sol_resin,barrier_parameters)
else:
  print("Not solution could be found for the selected Food Type. \n Try changing the food type, and run again.")

['LDPE', 'LLDPE', 'MDPE', 'ULDPE', 'HDPE', 'mPE', 'COC', 'PP', 'BOPP', 'PC', 'BOPET', 'PA6', 'PA66', 'PU', 'PS', 'PLA', 'Cellulose']
No material available
Not solution could be found for the selected FoodType. 
 Try changing the food type :)
